## Installing Libraries

In [1]:
!pip install langchain langchain-community langchain-core faiss-cpu pypdf2 pyngrok sentence-transformers transformers==4.52.4

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 79.3 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 58.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 88.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: packaging
    Found existing installation: packaging 26.0rc2
    Uninstalling packaging-26.0rc2:
      Successfully uninstalled packaging-26.0rc2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Atte

## Importing Libraries

In [31]:
from huggingface_hub import login
from transformers import pipeline, set_seed
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms.base import LLM
from typing import Any
from langchain.chains import LLMChain
import torch
import re
from langchain.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader
import io
from fastapi import FastAPI, Request, HTTPException
import uvicorn, threading, time, socket
from pyngrok import ngrok, conf
import base64
from langchain.schema import Document

## Hugging Face Token

## Models from Hagging Face

In [3]:
QA_model = "mistralai/Mistral-Nemo-Instruct-2407"
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"

In [4]:
tokenizer_QA = AutoTokenizer.from_pretrained(QA_model)
model = AutoModelForCausalLM.from_pretrained(QA_model, torch_dtype=torch.float16, device_map="auto")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

## Reading PDF using LangChain

In [10]:
pdf_path = "/kaggle/input/scientific-paper/Plant_Disease_Classification.pdf"

In [8]:
def extract_pdf(path):
    '''this function aims to load, extract, split, embed and create vector database using langchain tools'''
    loader = PyPDFLoader(path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, separators=["\n\n", "\n", ". ", " ", ""])
    chunks = text_splitter.split_documents(documents)
    embedding = HuggingFaceEmbeddings(model_name=embedding_model)
    vectordb = FAISS.from_documents(chunks, embedding)
    return documents, chunks, embedding, vectordb

In [32]:
def prepare_pdf(text):
    '''this function aims to split, embed and create vector database using langchain tools used in deployment'''
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, separators=["\n\n", "\n", ". ", " ", ""])
    doc = Document(page_content=text)
    ##convert the output text into document to split it using lamgchin splitter
    chunks = text_splitter.split_documents([doc])
    embedding = HuggingFaceEmbeddings(model_name=embedding_model)
    vectordb = FAISS.from_documents(chunks, embedding)
    return chunks, embedding, vectordb

In [11]:
documents, chunks, embedding, vectordb = extract_pdf(pdf_path)

/tmp/ipykernel_55/2586350219.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=embedding_model)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
final_text = "\n".join([doc.page_content for doc in documents])
print(final_text)

Received 15 September 2023, accepted 4 October 2023, date of publication 16 October 2023, date of current version 20 October 2023.
Digital Object Identifier 10.1 109/ACCESS.2023.3324722
Machine Learning and Deep Learning for Plant
Disease Classification and Detection
VASILEIOS BALAFAS
 , EMMANOUIL KARANTOUMANIS
 ,
MALAMATI LOUTA
 , (Senior Member, IEEE),
AND NIKOLAOS PLOSKAS
Department of Electrical and Computer Engineering, University of Western Macedonia, 50100 Kozani, Greece
Corresponding author: Nikolaos Ploskas (nploskas@uowm.gr)
This work was supported by the European Regional Development Fund of the European Union and Greek national funds through the
Operational Program Competitiveness, Entrepreneurship and Innovation under Project MIS 5047196. The publication of the article in OA
mode was financially supported by HEAL-Link.
ABSTRACT Precision agriculture is a rapidly developing field aimed at addressing current concerns
about agricultural sustainability. Machine learning is the

## **Feature1:** Research based Question Answering
**Using Output Parser for structured output**

In [13]:
##to prevent any hallucination in the answer of the question, we removed (temprature, top_k, top_p...)
def generate_answer(prompt, max_new_tokens=512):    #max_new to control output independent from input
    inputs = tokenizer_QA(prompt,return_tensors="pt", truncation=True).to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,      
        num_beams=1,          
        early_stopping=True
    )
    return tokenizer_QA.decode(outputs[0], skip_special_tokens=True)

In [14]:
Question_schema = ResponseSchema(
    name = "Question",
    description = "The question took from user input."
)

Answer_schema = ResponseSchema(
    name = "Answer",
    description = "the Generated answer from the model."
)

Evidence_schema = ResponseSchema(
    name = "Evidence",
    description = "list of string containing provided 1-3 exact quotes from the context as evidence, if non, return empty list."
)

Pages_schema = ResponseSchema(
    name = "Pages",
    description = "list of integers that refere to page number containing the answer of user question."
)

response_schema = [Question_schema,Answer_schema,Evidence_schema,Pages_schema]

In [15]:
output_parser = StructuredOutputParser.from_response_schemas(response_schema)
format_instructions = output_parser.get_format_instructions()

In [16]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Question": string  // The question took from user input.
	"Answer": string  // the Generated answer from the model.
	"Evidence": string  // list of string containing provided 1-3 exact quotes from the context as evidence, if non, return empty list.
	"Pages": string  // list of integers that refere to page number containing the answer of user question.
}
```


In [17]:
purchase_extraction_template = """
You must answer using ONLY the context below.
Rules:
- Do NOT use outside knowledge.
- Do NOT explain beyond what the context says.
- If the answer is not explicitly stated in the context, set "answer" to "Not mentioned" and "evidence" to [].
- If the answer is a list: list ALL items explicitly.
- After the answer, provide 1-3 exact quotes from the context as evidence.

Respond ONLY in JSON format as follows:
{format_instructions}

Context:
{context}

Now answer the following question:
"{query}"
"""

In [33]:
def Question_Answering(query):
    docs = vectordb.similarity_search(query, k=6)
    context = "\n\n".join([doc.page_content for doc in docs])
    
    prompt = PromptTemplate(
        template=purchase_extraction_template,
        input_variables=["context","query","format_instructions"]
    ).format(context =context ,query =query , format_instructions=format_instructions)
    
    answer = generate_answer(prompt, max_new_tokens=512)

    pattern = r'```json\s*(.*?)\s*```'
    matches = re.findall(pattern, answer, re.DOTALL)

    json_output = f"```json\n{matches[-1]}\n```" if matches else None

    ##set to prevent repitition in the pages
    pages = sorted(set(
        [doc.metadata.get('page', 'Unknown') for doc in docs]
    ))
    return json_output.strip(), pages

In [19]:
Question = 'According to the paper’s computational study on the PlantDoc dataset, which object detection model achieved the highest accuracy, and which classification networks had the best trade-off between accuracy and training time?'
result, page_number = Question_Answering(Question)
print("answer:", result)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


answer: ```json
{
	"Question": "According to the paper’s computational study on the PlantDoc dataset, which object detection model achieved the highest accuracy, and which classification networks had the best trade-off between accuracy and training time?",
	"Answer": "The object detection model that achieved the highest accuracy was YOLOv5. For the image classification task, the networks ResNet50 and MobileNetv2 had the most optimal trade-off on accuracy and training time.",
	"Evidence": ["For the object detection problem, computational results show that object detection accuracy is high with YOLOv5.", "For the image classification task, the networks ResNet50 and MobileNetv2 have the most optimal trade-off on accuracy and training time."],
	"Pages": [1, 2]
}
```


## Q/A Deplyment

In [22]:
NGROK_TOKEN = "386REcz4JsU2OpwGtAeOqsd8H7j_81puHo7ELtjbx6QxVhpZp"
API_KEY = "secret123"

In [23]:
def extract_text_from_bytes(pdf_bytes):
    pdf_file = io.BytesIO(pdf_bytes)
    reader = PdfReader(pdf_file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

In [35]:
def extract_json_block(text):
    pattern = r'```json\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)
    return matches[-1].strip() if matches else text.strip()

## **Feature3:** Basic Information Extraction

In [50]:
def extract_final_answer(text: str) -> str:
    if "Answer:" in text:
        return text.split("Answer:")[-1].strip()
    return text.strip()

In [ ]:
def Question_Answering_chain(query):
    docs = vectordb.similarity_search(query, k=6)
    context = "\n\n".join([doc.page_content for doc in docs])

    prompt = f"""
            Answer ONLY using the context below.
            Return ONLY the final answer.
            Do NOT explain.
            Do NOT repeat the context.
            
            Context:
            {context}
            
            Question:
            {query}
            """

    answer = generate_answer(prompt, max_new_tokens=512)
    return extract_final_answer(answer)

In [52]:
##make open source as closed source
class CustomHFLLM(LLM):
    def _call(self, prompt: str, stop: Any = None) -> str:
        return Question_Answering_chain(prompt)
    
    @property
    def _llm_type(self) -> str:
        return "custom_huggingface"

llm = CustomHFLLM()

In [53]:
# Chain 1: problem statement
problem_prompt = PromptTemplate(
    input_variables=["question"],
    template="Extract from this paper the problem statement,Return ONLY the main research problem in 2-3 sentences"
)
problem_chain = LLMChain(llm=llm, prompt=problem_prompt)

# Chain 2: Datasets
dataset_prompt = PromptTemplate(
    input_variables=["question"],
    template='''List all the datasets used in this paper, for each dataset mention its name, 
                number of images and the classes it include.
                Return ONLY JSON format:
                [{{"name": "PlantDoc", "images": 5000, "classes": ["disease1", "disease2"]}}]'''
)
datasete_chain = LLMChain(llm=llm, prompt=dataset_prompt)

# Chain 3: Models
model_prompt = PromptTemplate(
    input_variables=["question"],
    template='''List ALL ML/DL MODELS used in this paper.
                Include: CNNs, object detectors, classifiers.
                Return ONLY JSON: [{{"name": "ResNet50", "type": "CNN", "task": "classification"}}]'''
)
model_chain = LLMChain(llm=llm, prompt=model_prompt)

# Chain 4: Results
result_prompt = PromptTemplate(
    input_variables=["question"],
    template=("List the metrics used to evaluate the model performance and list the final results of the work provided in the paper.")
)
result_chain = LLMChain(llm=llm, prompt=result_prompt)

/tmp/ipykernel_55/3364785747.py:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  problem_chain = LLMChain(llm=llm, prompt=problem_prompt)


In [79]:
def run_all():
    problem = problem_chain.run(question= "What is the main research problem?")
    print(f"\nProblem Statement:\n{problem}")

    dataset = datasete_chain.run(question= "What datasets are used in the paper?")
    print(f"\nDatasets:\n{dataset}")

    models = model_chain.run(question = "What models are used in the paper?")
    print(f"\nModels:\n{models}")

    results = result_chain.run(question= "What are the evaluation metrics and final results?")
    print(f"\nFinal Results:\n{results}")


In [80]:
run_all()

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Problem Statement:
The main research problem is the development of machine learning algorithms for plant disease detection and classification, focusing on distinguishing between healthy plants and those with diseases such as common rust, Northern blight, and multiple diseases in a single image.


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Datasets:
[
                {"name": "PlantLeaves", "images": 4503, "classes": 22},
                {"name": "PlantDoc", "images": 5000, "classes": 15},
                {"name": "PlantVillage", "images": 87848, "classes": 58}
            ]


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Models:
[
                {"name": "ResNet50", "type": "CNN", "task": "classification"},
                {"name": "VGG16", "type": "CNN", "task": "classification"},
                {"name": "InceptionV3", "type": "CNN", "task": "classification"},
                {"name": "DenseNet121", "type": "CNN", "task": "classification"},
                {"name": "GoogleNet", "type": "CNN", "task": "classification"},
                {"name": "AlexNet", "type": "CNN", "task": "classification"},
                {"name": "DenseNet121", "type": "CNN", "task": "classification"},
                {"name": "DenseNet161", "type": "CNN", "task": "classification"},
                {"name": "DenseNet169", "type": "CNN", "task": "classification"},
                {"name": "DenseNet201", "type": "CNN", "task": "classification"},
                {"name": "MobileNetV2", "type": "CNN", "task": "classification"},
                {"name": "ResNet50", "type": "CNN", "task": "classification"},
                {"name"

In [54]:
def run_all_api():
    problem = problem_chain.run(question="What is the main research problem?")

    dataset = datasete_chain.run(question="What datasets are used in the paper?")

    models = model_chain.run(question="What models are used in the paper?")

    results = result_chain.run(question="What are the evaluation metrics and final results?")
    return {
        "problem_statement": problem,
        "datasets": dataset,
        "models": models,
        "results": results
    }

## Features Deployment

In [59]:
app = FastAPI()

@app.post("/QA")
async def qa_endpoint(req: Request):
    if req.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")
    data = await req.json()
    pdf = data.get("pdf_file")
    pdf_bytes = base64.b64decode(pdf)
    pdf_text = extract_text_from_bytes(pdf_bytes)
    chunks, embedding, vectordb = prepare_pdf(pdf_text)
    question = data.get("question")
    raw_json, pages = Question_Answering(question)
    json_text = extract_json_block(raw_json)
    output_data = output_parser.parse(json_text)
    return {"answer": {**output_data, "Pages": pages}}

# --- Info Extractor Endpoint ---
@app.post("/info_Extractor")
async def extractor_endpoint(req: Request):
    if req.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")
    data = await req.json()
    pdf = data.get("pdf_file")
    pdf_bytes = base64.b64decode(pdf)
    pdf_text = extract_text_from_bytes(pdf_bytes)
    prepare_pdf(pdf_text)
    output = run_all_api()
    return output


In [60]:
def free_port():
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

port = free_port()
conf.get_default().auth_token = NGROK_TOKEN
public_url = ngrok.connect(port).public_url
print("Your public URL:", public_url)

def run(): uvicorn.run(app, host="0.0.0.0", port=port)
threading.Thread(target=run, daemon=True).start()
time.sleep(1)

Your public URL: https://pennie-sabulous-rheba.ngrok-free.dev


INFO:     Started server process [55]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:60507 (Press CTRL+C to quit)
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     41.199.138.164:0 - "POST /QA HTTP/1.1" 200 OK


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     41.199.138.164:0 - "POST /info_Extractor HTTP/1.1" 200 OK


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     41.199.138.164:0 - "POST /QA HTTP/1.1" 200 OK
